In [ ]:
import logging
import os
import pandas as pd
import psycopg2

In [ ]:
def open_connection():
    conn = \
        psycopg2.connect( \
            host = 'localhost',
            database = 'queroponto',
            user = 'postgres',
            password = 'postgres')

    cur = conn.cursor()
    
    return conn, cur

In [ ]:
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [ ]:
def insert_data(conn, cur, table, id_sector, data):
    col_names = ', '.join([f"\"{item[0]}\"" for item in data])
    col_values = ['NULL' if item[1] != item[1] else item[1] for item in data]
    col_values = ', '.join(map(str, [item for item in col_values]))
    
    query = f"INSERT INTO public.{table} (id_setor_censitario, {col_names}) VALUES ({id_sector}, {col_values});"
    
    try:
        cur.execute(query)
    except psycopg2.IntegrityError as e:
        conn.rollback()
        # logger.error(f"Failed: {e}")
    else:
        conn.commit()

---

In [ ]:
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)s %(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [ ]:
# file = '../arquivos/csv/AC.csv' # 12 - OK
# file = '../arquivos/csv/AL.csv' # 27 - *
# file = '../arquivos/csv/AM.csv' # 13 - OK
# file = '../arquivos/csv/AP.csv' # 16 - OK
file = '../arquivos/csv/sociodemog_BA (1).csv' # 29 - *
# file = '../arquivos/csv/CE.csv' # 23 - OK
# file = '../arquivos/csv/DF.csv' # 53 - OK
# file = '../arquivos/csv/ES.csv' # 32 - OK
# file = '../arquivos/csv/GO.csv' # 52 - OK
# file = '../arquivos/csv/MA.csv' # 21 - OK
# file = '../arquivos/csv/MG.csv' # 31 - OK
# file = '../arquivos/csv/MS.csv' # 50 - *
# file = '../arquivos/csv/MT.csv' # 51 - OK
# file = '../arquivos/csv/PA.csv' # 15 - OK
# file = '../arquivos/csv/PB.csv' # 25 - *
# file = '../arquivos/csv/PE.csv' # 26 - *
# file = '../arquivos/csv/PI.csv' # 22 - *
# file = '../arquivos/csv/PR.csv' # 41 - OK
# file = '../arquivos/csv/RJ.csv' # 33 - OK
# file = '../arquivos/csv/RN.csv' # 24 - OK
# file = '../arquivos/csv/RO.csv' # 11 - OK
# file = '../arquivos/csv/RR.csv' # 14 - OK
# file = '../arquivos/csv/RS.csv' # 43 - *
# file = '../arquivos/csv/SC.csv' # 42 - ?
# file = '../arquivos/csv/SE.csv' # 28 - *
# file = '../arquivos/csv/SP.csv' # 35 - OK
# file = '../arquivos/csv/TO.csv' # 17 - OK

In [ ]:
# # Male
# cols = [
#     {"00_04": [
#         "P11_V024", "P11_V025", "P11_V026", "P11_V025", "P11_V026", "P11_V027", 
#         "P11_V028", "P11_V029", "P11_V030", "P11_V031", "P11_V032", "P11_V033", 
#         "P11_V034", "P11_V035", "P11_V036", "P11_V037", "P11_V038"
#     ]},
#     {"05_09": ["P11_V039", "P11_V040", "P11_V041", "P11_V042", "P11_V043"]},
#     {"10_14": ["P11_V044", "P11_V045", "P11_V046", "P11_V047", "P11_V048"]},
#     {"15_19": ["P11_V049", "P11_V050", "P11_V051", "P11_V052", "P11_V053"]},
#     {"20_24": ["P11_V054", "P11_V055", "P11_V056", "P11_V057", "P11_V058"]},
#     {"25_29": ["P11_V059", "P11_V060", "P11_V061", "P11_V062", "P11_V063"]},
#     {"30_34": ["P11_V064", "P11_V065", "P11_V066", "P11_V067", "P11_V068"]},
#     {"35_39": ["P11_V069", "P11_V070", "P11_V071", "P11_V072", "P11_V073"]},
#     {"40_44": ["P11_V074", "P11_V075", "P11_V076", "P11_V077", "P11_V078"]},
#     {"45_49": ["P11_V079", "P11_V080", "P11_V081", "P11_V082", "P11_V083"]},
#     {"50_54": ["P11_V084", "P11_V085", "P11_V086", "P11_V087", "P11_V088"]},
#     {"55_59": ["P11_V089", "P11_V090", "P11_V091", "P11_V092", "P11_V093"]},
#     {"60_64": ["P11_V094", "P11_V095", "P11_V096", "P11_V097", "P11_V098"]},
#     {"65_69": ["P11_V099", "P11_V100", "P11_V101", "P11_V102", "P11_V103"]},
#     {"70_74": ["P11_V104", "P11_V105", "P11_V106", "P11_V107", "P11_V108"]},
#     {"75_79": ["P11_V109", "P11_V110", "P11_V111", "P11_V112", "P11_V113"]},
#     {"80_84": ["P11_V114", "P11_V115", "P11_V116", "P11_V117", "P11_V118"]},
#     {"85_89": ["P11_V119", "P11_V120", "P11_V121", "P11_V122", "P11_V123"]},
#     {"90_94": ["P11_V124", "P11_V125", "P11_V126", "P11_V127", "P11_V128"]},
#     {"95_99": ["P11_V129", "P11_V130", "P11_V131", "P11_V132", "P11_V133"]},
#     {"100": ["P11_V134"]}
# ]

In [ ]:
# Female
cols = [
    {"00_04": [
        "P12_V024", "P12_V025", "P12_V026", "P12_V025", "P12_V026", "P12_V027", 
        "P12_V028", "P12_V029", "P12_V030", "P12_V031", "P12_V032", "P12_V033", 
        "P12_V034", "P12_V035", "P12_V036", "P12_V037", "P12_V038"
    ]},
    {"05_09": ["P12_V039", "P12_V040", "P12_V041", "P12_V042", "P12_V043"]},
    {"10_14": ["P12_V044", "P12_V045", "P12_V046", "P12_V047", "P12_V048"]},
    {"15_19": ["P12_V049", "P12_V050", "P12_V051", "P12_V052", "P12_V053"]},
    {"20_24": ["P12_V054", "P12_V055", "P12_V056", "P12_V057", "P12_V058"]},
    {"25_29": ["P12_V059", "P12_V060", "P12_V061", "P12_V062", "P12_V063"]},
    {"30_34": ["P12_V064", "P12_V065", "P12_V066", "P12_V067", "P12_V068"]},
    {"35_39": ["P12_V069", "P12_V070", "P12_V071", "P12_V072", "P12_V073"]},
    {"40_44": ["P12_V074", "P12_V075", "P12_V076", "P12_V077", "P12_V078"]},
    {"45_49": ["P12_V079", "P12_V080", "P12_V081", "P12_V082", "P12_V083"]},
    {"50_54": ["P12_V084", "P12_V085", "P12_V086", "P12_V087", "P12_V088"]},
    {"55_59": ["P12_V089", "P12_V090", "P12_V091", "P12_V092", "P12_V093"]},
    {"60_64": ["P12_V094", "P12_V095", "P12_V096", "P12_V097", "P12_V098"]},
    {"65_69": ["P12_V099", "P12_V100", "P12_V101", "P12_V102", "P12_V103"]},
    {"70_74": ["P12_V104", "P12_V105", "P12_V106", "P12_V107", "P12_V108"]},
    {"75_79": ["P12_V109", "P12_V110", "P12_V111", "P12_V112", "P12_V113"]},
    {"80_84": ["P12_V114", "P12_V115", "P12_V116", "P12_V117", "P12_V118"]},
    {"85_89": ["P12_V119", "P12_V120", "P12_V121", "P12_V122", "P12_V123"]},
    {"90_94": ["P12_V124", "P12_V125", "P12_V126", "P12_V127", "P12_V128"]},
    {"95_99": ["P12_V129", "P12_V130", "P12_V131", "P12_V132", "P12_V133"]},
    {"100": ["P12_V134"]}
]

In [ ]:
base_cols = []

for col in cols:
    for base in col.values():
        base_cols.append(base)
        
base_cols = [col for cols in base_cols for col in cols]
base_cols.insert(0, 'Cod_setor')

In [ ]:
%%time
df = pd.read_csv(file, usecols = base_cols, sep = ';', low_memory = False)

In [ ]:
logger.info(f"{file.split('/')[-1]} with {len(df)} rows.")

In [ ]:
%%time
df = df.apply(pd.to_numeric, errors = 'coerce')

In [ ]:
df.head()

In [ ]:
conn, cur = open_connection()

In [ ]:
%%time
for index, row in df.iterrows():
    data = []
    
    for col in cols:
        for key, values in col.items():
            values = [float(row[value]) for value in values]
            result = sum(values)
            data.append([key, result])
    
    # Male
    # insert_data(conn, cur, 'populacao_residente_mas', int(row['Cod_str']), data)
    
    # Female
    insert_data(conn, cur, 'populacao_residente_fem', int(row['Cod_setor']), data)

In [ ]:
close_connection(conn, cur)